In [1]:
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
from torch.optim import Adam
import numpy as np
import gymnasium as gym
from gymnasium.spaces import Discrete, Box
from copy import deepcopy


In [2]:
def mlp(sizes, activation=nn.Tanh, output_activation=nn.Identity):
    # Build a feedforward neural network.
    layers = []
    for j in range(len(sizes)-1):
        act = activation if j < len(sizes)-2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j+1]), act()]
    return nn.Sequential(*layers)


In [3]:
# make environment, check spaces, get obs / act dims
env_name = "LunarLander-v3" 
env = gym.make(env_name)
#env = gym.make(env_name, render_mode="human")
assert isinstance(env.observation_space, Box), \
    "This example only works for envs with continuous state spaces."
assert isinstance(env.action_space, Discrete), \
    "This example only works for envs with discrete action spaces."

obs_dim = env.observation_space.shape[0]
print(f"Observation dimension: {obs_dim}")
print(f"Action dimension: {env.action_space.n}")
n_acts = env.action_space.n
hidden_sizes=[32]
logits_net = mlp(sizes=[obs_dim]+hidden_sizes+[n_acts])


Observation dimension: 8
Action dimension: 4


/home/mb600l/Documents/Projects/UltimateMLCourse/GenAI + Pytorch/Transformer/huggingface-LLM-RL/.venv/lib/python3.10/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [4]:
def train(env, lr=1e-3, 
          epochs=10_000, batch_size=50000, render=True):
    
    # make function to compute action distribution
    def get_policy(obs):
        logits = logits_net(obs)
        return Categorical(logits=logits)

    # make action selection function (outputs int actions, sampled from policy)
    def get_action(obs):
        return get_policy(obs).sample().item()

    # make loss function whose gradient, for the right data, is policy gradient
    def compute_loss(obs, act, weights):
        logp = get_policy(obs).log_prob(act)
        return -(logp * weights).mean()

    # make optimizer
    optimizer = Adam(logits_net.parameters(), lr=lr)

    # for training policy
    def train_one_epoch():
        # make some empty lists for logging.
        batch_obs = []          # for observations
        batch_acts = []         # for actions
        batch_weights = []      # for R(tau) weighting in policy gradient
        batch_rets = []         # for measuring episode returns
        batch_lens = []         # for measuring episode lengths

        # reset episode-specific variables
        obs = env.reset()       # first obs comes from starting distribution
        obs = obs[0] if isinstance(obs, tuple) else obs  # handle gymnasium reset return
        done = False            # signal from environment that episode is over
        ep_rews = []            # list for rewards accrued throughout ep

        # render first episode of each epoch
        finished_rendering_this_epoch = False

        # collect experience by acting in the environment with current policy
        while True:

            # rendering
            if (not finished_rendering_this_epoch) and render:
                env.render()

            # save obs
            #print(obs)
            batch_obs.append(deepcopy(obs))

            # act in the environment
            act = get_action(torch.as_tensor(obs, dtype=torch.float32))
            # observation, reward, terminated, truncated, info
            obs, rew, terminated, truncated, info = env.step(act)
            done = terminated or truncated

            # save action, reward
            batch_acts.append(act)
            ep_rews.append(rew)

            if done:
                # if episode is over, record info about episode
                ep_ret, ep_len = sum(ep_rews), len(ep_rews)
                batch_rets.append(ep_ret)
                batch_lens.append(ep_len)

                # the weight for each logprob(a|s) is R(tau)
                batch_weights += [ep_ret] * ep_len

                # reset episode-specific variables
                obs, done, ep_rews = env.reset(), False, []
                obs = obs[0] if isinstance(obs, tuple) else obs  # handle gymnasium reset return
                # won't render again this epoch
                finished_rendering_this_epoch = True

                # end experience loop if we have enough of it
                if len(batch_obs) > batch_size:
                    break

        # take a single policy gradient update step
        optimizer.zero_grad()
        batch_loss = compute_loss(obs=torch.as_tensor(batch_obs, dtype=torch.float32),
                                  act=torch.as_tensor(batch_acts, dtype=torch.int32),
                                  weights=torch.as_tensor(batch_weights, dtype=torch.float32)
                                  )
        batch_loss.backward()
        optimizer.step()
        return batch_loss, batch_rets, batch_lens

    # training loop
    for i in range(epochs):
        batch_loss, batch_rets, batch_lens = train_one_epoch()
        print('epoch: %3d \t loss: %.3f \t return: %.3f \t ep_len: %.3f'%
                (i, batch_loss, np.mean(batch_rets), np.mean(batch_lens)))



In [ ]:
train(env, render=True, lr=1e-3)

/home/mb600l/Documents/Projects/UltimateMLCourse/GenAI + Pytorch/Transformer/huggingface-LLM-RL/.venv/lib/python3.10/site-packages/gymnasium/envs/box2d/lunar_lander.py:672: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("LunarLander-v3", render_mode="rgb_array")
  gym.logger.warn(
/tmp/ipykernel_10626/2071510649.py:81: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch_loss = compute_loss(obs=torch.as_tensor(batch_obs, dtype=torch.float32),


epoch:   0 	 loss: -220.803 	 return: -154.240 	 ep_len: 85.538
epoch:   1 	 loss: -225.890 	 return: -166.350 	 ep_len: 89.815
epoch:   2 	 loss: -220.776 	 return: -154.393 	 ep_len: 85.985
epoch:   3 	 loss: -217.452 	 return: -156.356 	 ep_len: 87.796
epoch:   4 	 loss: -206.357 	 return: -149.375 	 ep_len: 86.814
epoch:   5 	 loss: -217.028 	 return: -157.030 	 ep_len: 86.988
epoch:   6 	 loss: -215.040 	 return: -151.541 	 ep_len: 84.249
epoch:   7 	 loss: -200.256 	 return: -149.669 	 ep_len: 88.553
epoch:   8 	 loss: -204.232 	 return: -148.755 	 ep_len: 85.948
epoch:   9 	 loss: -193.545 	 return: -141.342 	 ep_len: 85.625
epoch:  10 	 loss: -201.954 	 return: -143.229 	 ep_len: 84.361
epoch:  11 	 loss: -200.372 	 return: -142.018 	 ep_len: 83.543
epoch:  12 	 loss: -198.823 	 return: -140.448 	 ep_len: 84.657
epoch:  13 	 loss: -210.815 	 return: -148.982 	 ep_len: 83.896
epoch:  14 	 loss: -189.856 	 return: -141.597 	 ep_len: 86.807
epoch:  15 	 loss: -197.194 	 return: -1

In [ ]:
def test(env_name="LunarLander-v3", logits_net=logits_net, epochs=10, render=True):
    logits_net.eval()  # set logits_net to eval mode, so it doesn't track gradients

    # make environment, check spaces, get obs / act dims
    env = gym.make(env_name, render_mode="human")


    # make function to compute action distribution
    def get_policy(obs):
        logits = logits_net(obs)
        return Categorical(logits=logits)

    # make action selection function (outputs int actions, sampled from policy)
    def get_action(obs):
        return get_policy(obs).sample().item()

    # make loss function whose gradient, for the right data, is policy gradient
    def compute_loss(obs, act, weights):
        logp = get_policy(obs).log_prob(act)
        return -(logp * weights).mean()
    
    # for training policy
    def test_one_epoch():
        # make some empty lists for logging.
        batch_obs = []          # for observations
        batch_acts = []         # for actions
        batch_weights = []      # for R(tau) weighting in policy gradient
        batch_rets = []         # for measuring episode returns
        batch_lens = []         # for measuring episode lengths

        # reset episode-specific variables
        obs = env.reset()       # first obs comes from starting distribution
        obs = obs[0] if isinstance(obs, tuple) else obs  # handle gymnasium reset return
        done = False            # signal from environment that episode is over
        ep_rews = []            # list for rewards accrued throughout ep

        # render first episode of each epoch
        finished_rendering_this_epoch = False

        # collect experience by acting in the environment with current policy
        while True:

            # rendering
            if (not finished_rendering_this_epoch) and render:
                env.render()

            # save obs
            #print(obs)
            batch_obs.append(deepcopy(obs))

            # act in the environment
            act = get_action(torch.as_tensor(obs, dtype=torch.float32))
            # observation, reward, terminated, truncated, info
            obs, rew, terminated, truncated, info = env.step(act)
            done = terminated or truncated

            # save action, reward
            batch_acts.append(act)
            ep_rews.append(rew)

            if done:
                # if episode is over, record info about episode
                ep_ret, ep_len = sum(ep_rews), len(ep_rews)
                batch_rets.append(ep_ret)
                batch_lens.append(ep_len)

                # the weight for each logprob(a|s) is R(tau)
                batch_weights += [ep_ret] * ep_len

                # reset episode-specific variables
                obs, done, ep_rews = env.reset(), False, []
                obs = obs[0] if isinstance(obs, tuple) else obs  # handle gymnasium reset return
                # won't render again this epoch
                finished_rendering_this_epoch = True
                batch_loss = compute_loss(obs=torch.as_tensor(batch_obs, dtype=torch.float32),
                                  act=torch.as_tensor(batch_acts, dtype=torch.int32),
                                  weights=torch.as_tensor(batch_weights, dtype=torch.float32)
                                  )
                return batch_loss, batch_rets, batch_lens

    # testing loop
    r = 0
    for i in range(epochs):
        batch_loss, batch_rets, batch_lens = test_one_epoch()
        r = r + np.mean(batch_rets)
        print('epoch: %3d \t loss: %.3f \t return: %.3f \t ep_len: %.3f'%
                (i, batch_loss, np.mean(batch_rets), np.mean(batch_lens)))
    print(f"Average Total return over {epochs} epochs: {r/epochs}")



In [ ]:
test(env_name=env_name, logits_net=logits_net)